In [157]:
import os
import json
import re
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [158]:
dotenv.load_dotenv()

True

In [159]:
language_model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name=os.getenv("OPENAI_CHAT_MODEL_NAME"),
    temperature=float(os.getenv("OPENAI_CHAT_MODEL_TEMPERATURE")),
    max_tokens=int(os.getenv("OPENAI_CHAT_MODEL_MAX_TOKENS")),
)

In [160]:
data = {
    "events": [
        "Hiking",
        "Sprint Meeting",
        "RTW 1211",
    ],
    "weather":
        {
            "general": "few clouds",
            "is_dew": "False",
            "is_rain": "False",
            "is_snow": "False"
        },
    "daytime": "midday",
    "art_style": "oil painting"
}

In [161]:
examples = [
    {
        "data": {
            "events": [
                "Kickoff Meeting",
                "DE10CBA3",
                "Cycling Tour",
                "Band Practice",
            ],
            "weather": {
                "general": "sunny",
                "is_dew": "False",
                "is_rain": "False",
                "is_snow": "False"
            },
            "daytime": "morning",
            "art_style": "photograph"
        },
        "image_prompt": "A photograph of a person on a bicycle, driving on a road in the woods, on a sunny morning.",

    }
]

In [162]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "data: {data}"),
    ("ai", "image prompt: '{image_prompt}'")
])

In [163]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [164]:
final_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Please generate a prompt for an image generation AI that visualizes the provided data (in JSON format)."),
    few_shot_prompt,
    ("human", "data: {data}"),
    ("ai", "image prompt: ")
])

prompt = final_prompt.format(data=data)
print(prompt)

System: Please generate a prompt for an image generation AI that visualizes the provided data (in JSON format).
Human: data: {'events': ['Kickoff Meeting', 'DE10CBA3', 'Cycling Tour', 'Band Practice'], 'weather': {'general': 'sunny', 'is_dew': 'False', 'is_rain': 'False', 'is_snow': 'False'}, 'daytime': 'morning', 'art_style': 'photograph'}
AI: image prompt: 'A photograph of a person on a bicycle, driving on a road in the woods, on a sunny morning.'
Human: data: {'events': ['Hiking', 'Sprint Meeting', 'RTW 1211'], 'weather': {'general': 'few clouds', 'is_dew': 'False', 'is_rain': 'False', 'is_snow': 'False'}, 'daytime': 'midday', 'art_style': 'oil painting'}
AI: image prompt: 


In [165]:
chain = final_prompt | language_model | StrOutputParser()
response = chain.invoke({"data": data})

print(response)

'An oil painting of a group of people hiking up a hill under a sky with few clouds during midday.'


In [166]:
result = re.findall("'([^']+)'", response)[0]
print(result)

An oil painting of a group of people hiking up a hill under a sky with few clouds during midday.
